In [73]:
import time
import pandas as pd
import numpy as np
from IPython.display import display

CITY_DATA = { 'chicago': 'chicago.csv',
              'new york city': 'new_york_city.csv',
              'washington': 'washington.csv' }
MONTHS = ['january', 'february', 'march', 'april', 'may', 'june', 'none']

DAYS =['monday', 'tuesday', 'wednesday', 'thursday',
        'friday', 'saturday', 'sunday', 'none']

def dayint_tostring(num):
    """
    This function takes an integer produced by the datetime module that
    represents a week day and returns the name of that week day as a string
    """
    mapp=['Monday', 'Tuesday', 'Wednesday', 'Thursday',
        'Friday', 'Saturday', 'Sunday']
    return mapp[num]

def monthint_tostring(num):
    """
    This function takes an integer produced by the datetime module
    that represents a month and returns the name of that month as a string
    """
    mapp=['January', 'February', 'March', 'April', 'May', 'June']
    return mapp[num-1]

In [ ]:
def get_filters():
    """
    Asks user to specify a city, month, and day to analyze.

    Returns:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by, or "all" to apply no day filter
        (book) True/False - whether the program should continue (True) or the user requested exit (False)
    """
    
    global CITY_DATA
    global MONTHS
    global DAYS
    print('Hello! Let\'s explore some US bikeshare data! Type exit anytime to shut down the program.')
        
    city = input('Which city would you like to see data for? Type "Chicago", "New York City, or "Washington".\n').lower().strip(' " \'')
    while True:
        if city == 'exit':
            return 0, 0, 0, False
        elif city in CITY_DATA:
            break
        else:
            city = input('This is not a valid city. \nType "Chicago", "New York City, "Washington", or "exit" to quit.\n').lower().strip(' " \'')
    
    month = input('To filter data by month, choose from (January, Febuary, March, April, May, June), or type none to ignore filter.\n').lower().strip(' " \'')
    while True:
        if month == 'exit':
            return 0, 0, 0, False
        elif month in MONTHS:
            break
        else:
            month = input('Invalid input. \nChoose from (January, Febuary, March, April, May, June) to filter by month, none for no filters, or "exit" to quit.\n').lower().strip(' " \'')
    
    day = input('To filter data by weekday, choose from (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday), or type none to ignore filter\n').lower().strip(' " \'')
    while True:
        if day == 'exit':
            return 0, 0, 0, False
        elif day in DAYS:
            break
        else:
            day = input('Invalid input. \nChoose from (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday) to filter by weekday, none for no filters, or "exit" to quit.\n').lower().strip(' " \'')

    print('-'*40)
    return city, month.title(), day.title(), True


def load_data(city, month, day):
    """
    Loads data for the specified city and filters by month and day if applicable.

    Args:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or none to apply no month filter
        (str) day - name of the day of week to filter by, or none to apply no day filter
    Returns:
        df - Pandas DataFrame containing city data filtered by month and day
    """
    
    global CITY_DATA
    city = CITY_DATA[city]
    
    df = pd.read_csv(city)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['Day of Week'] = df['Start Time'].dt.weekday
    df['Day of Week'] = df['Day of Week'].apply(dayint_tostring)
    df['Month'] = df['Start Time'].dt.month
    df['Month'] = df['Month'].apply(monthint_tostring)
    df['Hour'] = df['Start Time'].dt.hour

    df = df.drop(['Unnamed: 0'],axis=1)
    
    if month != 'None':
        df = df[df['Month'] ==month]
    
    if day != 'None':
        df = df[df['Day of Week'] ==day]
        
    print('Total trips in specified time is {}.'.format(df.shape[0]))
    return df



def time_stats(df):
    """Displays statistics on the most frequent times of travel."""

    print('\nCalculating The Most Frequent Times of Travel...\n')
    start_time = time.time()
    
    if len(df['Month'].unique()) > 1:
        highest = df['Month'].value_counts().idxmax()
        count = df['Month'].value_counts().max()
        print('The most popular month is {} with a total of {} rides.'.format(highest, count))

    if len(df['Day of Week'].unique()) > 1:
        highest = df['Day of Week'].value_counts().idxmax()
        count = df['Day of Week'].value_counts().max()
        print('The most popular day is {} with a total of {} rides.'.format(highest, count))
    
    highest = df['Hour'].value_counts().idxmax()
    count = df['Hour'].value_counts().max()
    
    if highest //12 ==0:
        clock = 'AM'
        if highest ==0:
            highest = 12
    else:
        clock = 'PM'
        if highest != 12:
            highest = highest % 12
    print('The most popular hour is {} {} with a total of {} rides.'.format(highest, clock, count))


    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


def station_stats(df):
    """Displays statistics on the most popular stations and trip."""

    print('\nCalculating The Most Popular Stations and Trip...\n')
    start_time = time.time()
    # display most commonly used start station
    highest = df['Start Station'].value_counts().idxmax()
    count = df['Start Station'].value_counts().max()
    print('The most popular starting station is {} with a total of {} rides.'.format(highest, count))
   

    # display most commonly used end station
    highest = df['End Station'].value_counts().idxmax()
    count = df['End Station'].value_counts().max()
    print('The most popular ending station is {} with a total of {} rides.'.format(highest, count))

    # display most frequent combination of start station and end station trip
    src, dest = df[['Start Station','End Station']].value_counts().idxmax()
    count = df[['Start Station','End Station']].value_counts().max()
    print('The most popular trip starts from {} and ends on {} with a total of {} trips.'.format(src, dest, count))
    
    
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


def trip_duration_stats(df):
    """Displays statistics on the total and average trip duration."""

    print('\nCalculating Trip Duration...\n')
    start_time = time.time()

    # display total travel time
    sumTT= df['Trip Duration'].sum()
    print('Total trip duration is {} seconds, or {} hours.'.format(sumTT, sumTT//3600))

    # display mean travel time
    meanTT= df['Trip Duration'].mean()
    print('Average trip duration is {} seconds, or {} minutes.'.format(meanTT, meanTT//60))
    
    
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


def user_stats(df):
    """Displays statistics on bikeshare users."""

    print('\nCalculating User Stats...\n')
    start_time = time.time()

    print('Out of a total of {} users:'.format(df.shape[0]))
    
    if 'User Type' in df.columns:
        for UserType in df['User Type'].unique():
            countoftype = df[df['User Type']==UserType].shape[0]
            print('There are {} users of type {}.'.format(countoftype, UserType))
    
    # Display counts of gender
    if 'Gender' in df.columns:
        print('\nOut of a total of {} Subscribers:'.format(df[df['User Type']=='Subscriber'].shape[0]))
        for gender in df['Gender'].unique():
            if type(gender) != str:
                continue
            countofgen = df[df['Gender']==gender].shape[0]
            print('There are {} users who are {}.'.format(countofgen, gender))
    # Display earliest, most recent, and most common year of birth
    if 'Birth Year' in df.columns:
        dob = df['Birth Year']
        print('\nThe earliest year of birth is {}, the most recent is {}, and the most common is {}.'.format(dob.min(), dob.max(), dob.mode()[0]))
    

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

def ShowData(city):
    """
    This function takes the city name as input to load the raw data again.
    It then prints 5 rows iteratively whenever the user types 'yes'. The function exits when 
    any command other than 'yes' is given. 
    """
    Display = input('Would you like to see 5 lines of raw data? Type "yes" or the program will exit.').lower().strip(' " \'')
    global CITY_DATA
    city = CITY_DATA[city]
    df = pd.read_csv(city)
    x = 0
    maximumx = df.shape[0]
    while Display == 'yes':
        display(df.loc[range(x,x+5)])
        Display = input('Would you like to see more?')
        x=x+5
        if x+5 > maximumx:
            display(df.loc[range(x,maximumx)])
            print('No more data to display.')
            break
        

In [81]:
def main():
    while True:
        city, month, day, ContinueProgram = get_filters()
        if ContinueProgram:
            df = load_data(city, month, day)
            time_stats(df)
            station_stats(df)
            trip_duration_stats(df)
            user_stats(df)
            ShowData(city)

        restart = input('\nWould you like to restart? Type "yes" or the program will exit.\n').lower().strip(' " \'')
        if restart.lower() != 'yes':
            break


if __name__ == "__main__":
	main()

Hello! Let's explore some US bikeshare data! Type exit anytime to shut down the program.


Which city would you like to see data for? Type "Chicago", "New York City, or "Washington".
 washington
To filter data by month, choose from (January, Febuary, March, April, May, June), or type none to ignore filter.
 june
To filter data by weekday, choose from (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday), or type none to ignore filter
 none


----------------------------------------
Total trips in specified time is 68339.

Calculating The Most Frequent Times of Travel...

The most popular day is Thursday with a total of 11727 rides.
The most popular hour is 8 AM with a total of 8878 rides.

This took 0.031239986419677734 seconds.
----------------------------------------

Calculating The Most Popular Stations and Trip...

The most popular starting station is Lincoln Memorial with a total of 1314 rides.
The most popular ending station is Jefferson Dr & 14th St SW with a total of 1339 rides.
The most popular trip starts from Jefferson Dr & 14th St SW and ends on Jefferson Dr & 14th St SW with a total of 190 trips.

This took 0.09933996200561523 seconds.
----------------------------------------

Calculating Trip Duration...

Total trip duration is 93658035.74599999 seconds, or 26016.0 hours.
Average trip duration is 1370.4917506255642 seconds, or 22.0 minutes.

This took 0.0 seconds.
----------------------------------------

Ca

Would you like to see 5 lines of raw data? Type "yes" or the program will exit. yes


,Unnamed: 0,Start Time,End Time,Trip Duration,Start Station,End Station,User Type
0,1621326,2017-06-21 08:36:34,2017-06-21 08:44:43,489.066,14th & Belmont St NW,15th & K St NW,Subscriber
1,482740,2017-03-11 10:40:00,2017-03-11 10:46:00,402.549,Yuma St & Tenley Circle NW,Connecticut Ave & Yuma St NW,Subscriber
2,1330037,2017-05-30 01:02:59,2017-05-30 01:13:37,637.251,17th St & Massachusetts Ave NW,5th & K St NW,Subscriber
3,665458,2017-04-02 07:48:35,2017-04-02 08:19:03,1827.341,Constitution Ave & 2nd St NW/DOL,M St & Pennsylvania Ave NW,Customer
4,1481135,2017-06-10 08:36:28,2017-06-10 09:02:17,1549.427,Henry Bacon Dr & Lincoln Memorial Circle NW,Maine Ave & 7th St SW,Subscriber


...

Would you like to see more? yes


,Unnamed: 0,Start Time,End Time,Trip Duration,Start Station,End Station,User Type
10,854729,2017-04-17 02:50:51,2017-04-17 03:05:17,866.689,37th & O St NW / Georgetown University,37th & O St NW / Georgetown University,Customer
11,721887,2017-04-08 01:12:48,2017-04-08 01:52:33,2385.411,10th & G St NW,5th St & Massachusetts Ave NW,Customer
12,143181,2017-01-26 20:01:00,2017-01-26 20:17:00,951.457,10th St & L'Enfant Plaza SW,1st & Rhode Island Ave NW,Subscriber
13,26953,2017-01-06 17:36:00,2017-01-06 17:43:00,448.628,North Capitol St & F St NW,1st & M St NE,Subscriber
14,1308542,2017-05-28 03:50:26,2017-05-28 04:24:26,2039.472,Henry Bacon Dr & Lincoln Memorial Circle NW,Lincoln Memorial,Customer


Would you like to see more? yes


,Unnamed: 0,Start Time,End Time,Trip Duration,Start Station,End Station,User Type
15,1592744,2017-06-19 07:46:35,2017-06-19 07:55:00,505.482,Metro Center / 12th & G St NW,North Capitol St & G Pl NE,Subscriber
16,1139713,2017-05-13 11:29:16,2017-05-13 11:55:47,1591.326,16th & Harvard St NW,New York Ave & 15th St NW,Customer
17,1478516,2017-06-10 06:04:41,2017-06-10 06:32:33,1672.075,17th St & Massachusetts Ave NW,15th & East Capitol St NE,Subscriber
18,679837,2017-04-03 10:22:19,2017-04-03 10:26:11,231.952,20th & Crystal Dr,6th & S Ball St,Subscriber
19,1392498,2017-06-03 11:42:54,2017-06-03 12:21:05,2291.454,22nd & I St NW / Foggy Bottom,Lamont & Mt Pleasant NW,Customer


Would you like to see more? no

Would you like to restart? Type "yes" or the program will exit.
 no
